In [93]:
import onnxruntime as ort
import cv2
import numpy as np
import os

session = ort.InferenceSession("/home/el02/PiPresence/data/models/yolov8n.onnx")
input_name = session.get_inputs()[0].name
image = cv2.imread("/home/el02/PiPresence/data/test_images/common.jpg")

# Preprocess image for ONNX model
print("[INFO] Preprocessing image for face detection")
# Resize image to the required size for YOLO model
input_image = cv2.resize(image, (640, 640))
# Normalize pixel values to range [0, 1]
input_image = input_image.astype(np.float32) / 255.0
# Change image layout to channel-first format as required by ONNX model
input_image = np.transpose(input_image, (2, 0, 1))  # Channel first
# Add batch dimension (needed by the model)
input_image = np.expand_dims(input_image, axis=0)

# Perform inference on the provided image
print("[INFO] Running inference on the image")
outputs = session.run(None, {input_name: input_image})
detections = outputs[0]
faces = []

# Iterate over detections and extract faces based on confidence threshold
for detection in detections:
    if detection[4] > 0.4:  # Confidence threshold
        print(f"[DEBUG] Detection confidence: {detection[4]} - Adding face to the list")
        # Extract bounding box coordinates
        x1, y1, x2, y2 = int(detection[0]), int(detection[1]), int(detection[2]), int(detection[3])
        # Crop the detected face from the original image
        face = image[y1:y2, x1:x2]
        faces.append(face)
    else:
        print(f"[DEBUG] Detection confidence too low: {detection[4]} - Ignoring")
print(faces, detection)

[INFO] Preprocessing image for face detection
[INFO] Running inference on the image


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import numpy as np

# Assuming outputs[0] has a shape of (1, 84, 8400)
output_tensor = outputs[0][0]  # Shape: (1, 84, 8400)
output_tensor = np.squeeze(output_tensor)  # Remove batch dimension, Shape: (84, 8400)

# Extract bounding box predictions (first 4 values)
bboxes = output_tensor[:4, :]  # Shape: (4, 8400)

# Extract objectness scores (5th value, index 4)
objectness_scores = output_tensor[4, :]  # Shape: (8400,)

# Extract class scores (remaining 80 values, from index 5 to 84)
class_scores = output_tensor[5:, :]  # Shape: (80, 8400)

# Compute the maximum class score for each prediction
max_class_scores = np.max(class_scores, axis=0)  # Shape: (8400,)

# Compute the final confidence score for each of the 8400 predictions
confidence_scores = objectness_scores * max_class_scores  # Shape: (8400,)

confidence_threshold = 0.5
selected_indices = confidence_scores > confidence_threshold

# Filtered bounding boxes, confidence scores, and class indices
filtered_bboxes = bboxes[:, selected_indices]  # Shape: (4, number of selected predictions)
filtered_confidences = confidence_scores[selected_indices]  # Shape: (number of selected predictions)
filtered_class_indices = np.argmax(class_scores[:, selected_indices], axis=0)  # Shape: (number of selected predictions)

In [101]:
filtered_class_indices

array([], dtype=int64)

In [74]:
import onnxruntime as ort
import cv2
import numpy as np

# Load the model and prepare image for inference
session = ort.InferenceSession("/home/el02/PiPresence/data/models/yolov8n.onnx")
input_name = session.get_inputs()[0].name
image = cv2.imread("/home/el02/PiPresence/data/test_images/common.jpg")

# Preprocess the image
print("[INFO] Preprocessing image for face detection")
input_image = cv2.resize(image, (640, 640))
input_image = input_image.astype(np.float32) / 255.0
input_image = np.transpose(input_image, (2, 0, 1))  # Channel-first format
input_image = np.expand_dims(input_image, axis=0)

# Perform inference
print("[INFO] Running inference on the image")
outputs = session.run(None, {input_name: input_image})
detections = outputs[0][0]  # Remove batch dimension

# Set confidence threshold
confidence_threshold = 0.4

# Iterate over detections and draw bounding boxes
for detection in detections:
    confidence = detection[4]
    if confidence > confidence_threshold:
        # Extract bounding box coordinates
        x1, y1, x2, y2 = int(detection[0]), int(detection[1]), int(detection[2]), int(detection[3])
        
        # Draw bounding box on the image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Draw the confidence score label
        label = f"Conf: {confidence:.2f}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        print(f"[INFO] Detection: Confidence {confidence:.2f}, Box: [{x1}, {y1}, {x2}, {y2}]")
    else:
        print(f"[DEBUG] Detection confidence too low: {confidence:.2f} - Ignoring")

# Show the resulting image with detections
cv2.imshow("Detections", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[INFO] Preprocessing image for face detection
[INFO] Running inference on the image
[INFO] Detection: Confidence 31.12, Box: [5, 22, 24, 26]
[INFO] Detection: Confidence 14.84, Box: [16, 15, 16, 16]
[INFO] Detection: Confidence 64.00, Box: [11, 45, 49, 53]
[INFO] Detection: Confidence 29.83, Box: [32, 31, 33, 33]
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low: 0.00 - Ignoring
[DEBUG] Detection confidence too low:

In [ ]:
from ultralytics import YOLO

# Load a YOLOv8 model from a .pt file
model = YOLO('../data/models/yolov8n.pt')  # Change 'yolov8n.pt' to your specific .pt file path

# Export the model to ONNX format
model.export(format='onnx', imgsz=(640, 640))  # Set imgsz to desired resolution, default is usually (640, 640)


100%|██████████| 6.25M/6.25M [00:00<00:00, 12.0MB/s]

Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.1+cu124 CPU (AMD Ryzen 9 7845HX with Radeon Graphics)


YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'data/models/yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.37...
ONNX: export success ✅ 0.4s, saved as 'data/models/yolov8n.onnx' (12.2 MB)

Export complete (0.6s)
Results saved to /home/el02/PiPresence/tests/data/models
Predict:         yolo predict task=detect model=data/models/yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=data/models/yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'data/models/yolov8n.onnx'

In [86]:
import onnx

# Load the exported ONNX model
onnx_model_path = 'data/models/yolov8n.onnx'
model = onnx.load(onnx_model_path)

# Check that the model is well formed
onnx.checker.check_model(model)
print("The model is correctly exported and valid.")


The model is correctly exported and valid.
